In [1]:
import pandas as pd
cars = pd.read_csv('Cars93.csv')
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Manufacturer        93 non-null     object 
 1   Model               93 non-null     object 
 2   Type                93 non-null     object 
 3   Min.Price           93 non-null     float64
 4   Price               93 non-null     float64
 5   Max.Price           93 non-null     float64
 6   MPG.city            93 non-null     int64  
 7   MPG.highway         93 non-null     int64  
 8   AirBags             93 non-null     object 
 9   DriveTrain          93 non-null     object 
 10  Cylinders           93 non-null     object 
 11  EngineSize          93 non-null     float64
 12  Horsepower          93 non-null     int64  
 13  RPM                 93 non-null     int64  
 14  Rev.per.mile        93 non-null     int64  
 15  Man.trans.avail     93 non-null     object 
 16  Fuel.tank.

In [3]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

#컬럼의 특수문자 제거
cars.columns = cars.columns.str.replace('.','')
model = smf.ols(formula='Price ~ EngineSize + RPM + Weight + Length + MPGcity + MPGhighway',data=cars)
result = model.fit()
result.summary()

C:\Users\suhye\AppData\Local\Temp\ipykernel_11188\3836818400.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cars.columns = cars.columns.str.replace('.','')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     19.14
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           4.88e-14
Time:                        16:19:54   Log-Likelihood:                -302.94
No. Observations:                  93   AIC:                             619.9
Df Residuals:                      86   BIC:                             637.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -32.2157     17.812     -1.809      0.074     -67.625       3.193
EngineSize     4.4732      1.410      3.172      0.002       1.670       7.276
RPM            0.0071      0.001      5.138      0.000       0.004       0.010
Weight         0.0056      0.003      1.634      0.106      -0.001       0.012
Length        -0.0464      0.094     -0.496      0.621      -0.232       0.139
MPGcity       -0.3478      0.448     -0.776      0.440      -1.239       0.544
MPGhighway     0.0582      0.460      0.126      0.900      -0.856       0.973
==============================================================================
Omnibus:                       62.984   Durbin-Watson:                   1.446
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              383.289
Skew:                           2.074   Prob(JB):                     5.89e-84
Kurtosis:                      12.039   Cond. No.                     1.61e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.61e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
#다중공선성 파악
#상관관계 0.9 이상이면 다중공선성이 존재한다고 판단

cars[['EngineSize','RPM','Weight','Length','MPGcity','MPGhighway']].corr()

,EngineSize,RPM,Weight,Length,MPGcity,MPGhighway
EngineSize,1.000000,-0.547898,0.845075,0.780283,-0.710003,-0.626795
RPM,-0.547898,1.000000,-0.427931,-0.441249,0.363045,0.313469
Weight,0.845075,-0.427931,1.000000,0.806274,-0.843139,-0.810658
Length,0.780283,-0.441249,0.806274,1.000000,-0.666239,-0.542897
MPGcity,-0.710003,0.363045,-0.843139,-0.666239,1.000000,0.943936
MPGhighway,-0.626795,0.313469,-0.810658,-0.542897,0.943936,1.000000


In [6]:
#VIF
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

#독립변수와 종속변수를 데이터프레임으로 나눠서 저장하는 함수
y,x = dmatrices('Price ~ EngineSize + RPM + Weight + Length + MPGcity + MPGhighway',
                data = cars, 
                return_type = 'dataframe')

vif_list = []
for i in range(1, len(x.columns)):
    vif_list.append([variance_inflation_factor(x.values, i),
                     x.columns[i]])
    
pd.DataFrame(vif_list, columns = ['vif','variable'])

,vif,variable
0,4.605118,EngineSize
1,1.446859,RPM
2,8.685973,Weight
3,4.013002,Length
4,13.668288,MPGcity
5,12.943133,MPGhighway


In [8]:
#MPGcity 변수 제거 후 다중 선형 회귀분석
model = smf.ols(formula='Price ~ EngineSize + RPM + Weight + MPGhighway',data=cars)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     28.98
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           2.26e-15
Time:                        16:27:19   Log-Likelihood:                -303.29
No. Observations:                  93   AIC:                             616.6
Df Residuals:                      88   BIC:                             629.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -37.7063     14.846     -2.540      0.013     -67.210      -8.203
EngineSize     4.5884      1.343      3.418      0.001       1.920       7.256
RPM            0.0071      0.001      5.232      0.000       0.004       0.010
Weight         0.0049      0.003      1.690      0.095      -0.001       0.011
MPGhighway    -0.2646      0.221     -1.200      0.234      -0.703       0.174
==============================================================================
Omnibus:                       62.072   Durbin-Watson:                   1.398
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              368.013
Skew:                           2.047   Prob(JB):                     1.22e-80
Kurtosis:                      11.843   Cond. No.                     1.35e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.35e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
import time
import itertools

def processSubset(x,y,feature_set):
    model = sm.OLS(y,x[list(feature_set)])
    regr = model.fit()
    AIC = regr.aic
    return {'model':regr, 'AIC':AIC}


#전진선택법
def forward(x,y, predictors):
    
    #데이터 변수들이 미리 정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in x.columns.difference(['Intercept']) if p not in predictors]
    
    results = []
    for p in remaining_predictors:
        results.append(processSubset(x=x,
                                     y=y,
                                     feature_set = predictors + [p] + ['Intercept']))
        
    #데이터프레임으로 변환
    models = pd.DataFrame(results)
        
    #AIC가 가장 낮은것 선택
    best_model = models.loc[models['AIC'].argmin()]
    print("Processed", models.shape[0],
          'models on', len(predictors)+1,
          'predictors in')
        
    print('Selected predictors:',best_model['model'].model.exog_names,
          'AIC:',best_model[0])
        
    return best_model
    
    
#후진소거법
def backward(x,y,predictors):
    tic = time.time()
    results = []
    
    #데이터 변수들이 미리 정의된 predictors 조합 확인
    for combo in itertools.combinations(predictors, len(predictors) -1):
        results.append(processSubset(x=x, y=y,
                                     feature_set = list(combo) +  ['Intercept']))
        
    models = pd.DataFrame(results)
    
    #가장 낮은 AIC를 가진 모델 선택
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    
    print('processed', models.shape[0],
          'models on', len(predictors) -1,
          'predictors in',(toc-tic))
    
    print('Selected predictors:',best_model['model'].model.exog_names,
          'AIC:',best_model[0])
    
    return best_model


#단계적 선택법
def Stepwise_model(x,y):
    
    Stepmodels = pd.DataFrame(columns=['AIC','model'])
    
    tic = time.time()
    predictors = []
    smodel_before = processSubset(x,y,predictors+['Intercept'])['AIC']
    
    for i in range(1, len(x.columns.difference(['Intercept']))+1):
        forward_result = forward(x=x,y=y,
                                 predictors=predictors)
        
        print('forward')
        Stepmodels.loc[i] = forward_result
        predictors = Stepmodels.loc[i]['model'].model.exog_names
        smodel_before = Stepmodels.loc[i]['AIC']
        predictors = [k for k in predictors if k!='Intercept']
        backward_result = backward(x=x,y=y,predictors=predictors)
        
        if backward_result['AIC'] < forward_result['AIC']:
            Stepmodels.loc[i] = backward_result
            predictors = Stepmodels.loc[i]['model'].model.exog_names
            smodel_before = Stepmodels.loc[i]["AIC"]
            predictors = [ k for k in predictors if k!= 'Intercept']
            print('backward')
        
        
        if Stepmodels.loc[i]['AIC'] > smodel_before:
            break
        else:
            smodel_before = Stepmodels.loc[i]['AIC']
            
    toc = time.time()
    print('total elapsed time: ',(toc-tic),'seconds.')
    return (Stepmodels['model'][len(Stepmodels['model'])])

In [27]:
stepwise_best_model = Stepwise_model(x=x,y=y)

Processed 6 models on 1 predictors in
Selected predictors: ['Weight', 'Intercept'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001DDA1739E20>
forward
processed 1 models on 0 predictors in 0.00559234619140625
Selected predictors: ['Intercept'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001DDA1722FD0>
Processed 5 models on 2 predictors in
Selected predictors: ['Weight', 'RPM', 'Intercept'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001DDA1739B20>
forward
processed 2 models on 1 predictors in 0.004472017288208008
Selected predictors: ['Weight', 'Intercept'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001DDA00BDF40>
Processed 4 models on 3 predictors in
Selected predictors: ['Weight', 'RPM', 'EngineSize', 'Intercept'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001DDA1739250>
forward
processed 3 model

In [28]:
stepwise_best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.547
Method:                 Least Squares   F-statistic:                     37.98
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           6.75e-16
Time:                        17:44:47   Log-Likelihood:                -304.05
No. Observations:                  93   AIC:                             616.1
Df Residuals:                      89   BIC:                             626.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Weight         0.0073      0.002      3.372      0.001       0.003       0.012
RPM            0.0071      0.001      5.208      0.000       0.004       0.010
EngineSize     4.3054      1.325      3.249      0.002       1.673       6.938
Intercept    -51.7933      9.106     -5.688      0.000     -69.887     -33.699
==============================================================================
Omnibus:                       62.441   Durbin-Watson:                   1.406
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              361.880
Skew:                           2.076   Prob(JB):                     2.62e-79
Kurtosis:                      11.726   Cond. No.                     8.27e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""